In [1]:
import os
import re
import requests
import sys
from num2words import num2words
import os
import pandas as pd
import numpy as np
import tiktoken
from openai import AzureOpenAI
import os
# importing necessary functions from dotenv library
from dotenv import load_dotenv, dotenv_values
import faiss
import numpy as np
import pymysql
import boto3
import pickle


 
bedrock_runtime = boto3.client(
    service_name='bedrock-runtime',
    region_name='us-east-1', 
    
)

In [2]:
load_dotenv(".env")


endpoint = os.getenv('RDS_ENDPOINT')
username = os.getenv('RDS_USERNAME')
password = os.getenv('RDS_PASSWORD')
database = os.getenv('RDS_DATABASE')

endpoint = "database-supermarket-smart.c9owiq2sebpi.us-east-1.rds.amazonaws.com"
username = "admin"
password = "Bb123456!"
database = "databasesupermarketsmart"

# Establish a connection to the RDS instance
connection = pymysql.connect(
    host=endpoint,
    user=username,
    password=password,
    database=database
)
cursor = connection.cursor()
# Execute a query
cursor.execute("SELECT VERSION()")
version = cursor.fetchone()
print(f"Database version: {version}")

Database version: ('8.0.35',)


In [3]:
select_sql = "SELECT recipe_id, recipe_name, instructions_and_ingredients FROM recipes;"
        
        # Execute the query
cursor.execute(select_sql)

# Fetch all results
results = cursor.fetchall()
print(results)
# Create a DataFrame from the results
df_recipes = pd.DataFrame(results, columns=['recipe_id', 'recipe_name', 'instructions_and_ingredients'])


select_sql = "SELECT grocery_type_id, grocery_type_name FROM groceries_types;"
        
        # Execute the query
cursor.execute(select_sql)

# Fetch all results
results = cursor.fetchall()
print(results)

# Create a DataFrame from the results
df_groceries = pd.DataFrame(results, columns=['grocery_type_id', 'grocery_type_name'])

((1, 'פאי תפוחים עם קרום פריך', 'מצרכים: תפוחים (4 יחידות), קמח (200 גרם), סוכר (100 גרם). הוראות הכנה: 1. מכינים את הבצק ומניחים בתבנית. 2. מכינים את מילוי התפוחים ומניחים מעל הבצק. 3. אופים בחום בינוני עד שהפאי זהוב ופריך.'), (2, 'מוס שוקולד מריר', 'מצרכים: שוקולד מריר (200 גרם), שמנת מתוקה (200 מ"ל), סוכר (50 גרם). הוראות הכנה: 1. ממיסים את השוקולד ושמים בצד. 2. מקציפים את השמנת עם הסוכר. 3. מערבבים את השוקולד עם השמנת המוקצפת. 4. מקררים לפני ההגשה.'), (3, 'סלט ירקות עם גבינה בולגרית', 'מצרכים: עגבניות (3 יחידות), מלפפונים (2 יחידות), גבינה בולגרית (100 גרם), שמן זית (2 כפות). הוראות הכנה: 1. חותכים את הירקות לחתיכות קטנות. 2. מוסיפים את הגבינה ואת שמן הזית. 3. מערבבים היטב ומגישים.'), (4, 'פסטה ברוטב עגבניות', 'מצרכים: פסטה (250 גרם), רוטב עגבניות (1 כוס), בצל (1 יחידה), שום (2 שיניים). הוראות הכנה: 1. מבשלים את הפסטה לפי ההוראות על האריזת. 2. מכינים את הרוטב על ידי טיגון הבצל והשום, הוספת רוטב העגבניות ובישול עוד כ-10 דקות. 3. מערבבים את הפסטה עם הרוטב ומגישים.'), (5, 'פנקייקים קל

In [4]:
import boto3
import json
 
#Create the connection to Bedrock


# Define prompt and model parameters
modelId = "amazon.titan-embed-text-v2:0"   

def generate_embeddings(prompt_data, modelId = "amazon.titan-embed-text-v2:0"):
    accept = "application/json"
    contentType = "application/json"
    sample_model_input={
        "inputText": prompt_data,
        "dimensions": 256,
        "normalize": True
    }

    body = json.dumps(sample_model_input)
    # Invoke model
    response = bedrock_runtime.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)

    response_body = json.loads(response.get('body').read())
    embedding = response_body.get("embedding")
    # Print response and embedding
    print(f"The embedding vector has {len(embedding)} values\n{embedding[0:3]+['...']+embedding[-3:]}")
    return np.array(embedding)

# Create the combined_info column
df_recipes['combined_info'] = df_recipes.apply(
    lambda row: row['recipe_name'] + ' - ' + row['instructions_and_ingredients'],
    axis=1
)


In [5]:
# Create the combined_info column
df_recipes['combined_info'] = df_recipes.apply(
    lambda row: row['recipe_name'],
    axis=1
)


In [6]:
df_recipes

,recipe_id,recipe_name,instructions_and_ingredients,combined_info
0,1,פאי תפוחים עם קרום פריך,"מצרכים: תפוחים (4 יחידות), קמח (200 גרם), סוכר...",פאי תפוחים עם קרום פריך
1,2,מוס שוקולד מריר,"מצרכים: שוקולד מריר (200 גרם), שמנת מתוקה (200...",מוס שוקולד מריר
2,3,סלט ירקות עם גבינה בולגרית,"מצרכים: עגבניות (3 יחידות), מלפפונים (2 יחידות...",סלט ירקות עם גבינה בולגרית
3,4,פסטה ברוטב עגבניות,"מצרכים: פסטה (250 גרם), רוטב עגבניות (1 כוס), ...",פסטה ברוטב עגבניות
4,5,פנקייקים קלאסיים,"מצרכים: קמח (200 גרם), חלב (300 מ""ל), ביצים (2...",פנקייקים קלאסיים
5,6,מרק ירקות בריא,"מצרכים: תפוח אדמה (2 יחידות), גזר (2 יחידות), ...",מרק ירקות בריא
6,7,שקשוקה עם ביצים,"מצרכים: ביצים (4 יחידות), עגבניות (3 יחידות), ...",שקשוקה עם ביצים
7,8,עוגת שוקולד קלה,"מצרכים: שוקולד מריר (200 גרם), חמאה (100 גרם),...",עוגת שוקולד קלה


In [7]:
df_recipes['embed_data'] = df_recipes['combined_info'].apply(lambda x : generate_embeddings (x)) # model should be set to the deployment name you chose when you deployed the text-embedding-ada-002 (Version 2) model
df_groceries['embed_data'] = df_groceries["grocery_type_name"].apply(lambda x : generate_embeddings (x)) # model should be set to the deployment name you chose when you deployed the text-embedding-ada-002 (Version 2) model

The embedding vector has 256 values
[-0.09596031904220581, 0.09596031904220581, 0.03850260004401207, '...', 0.007182215340435505, -0.05834624543786049, 0.0005599536816589534]
The embedding vector has 256 values
[-0.01313454657793045, 0.059526026248931885, -0.05357342213392258, '...', 0.06884314119815826, -0.0797131136059761, -0.00218369928188622]
The embedding vector has 256 values
[-0.08729195594787598, 0.0761185809969902, -0.030552184209227562, '...', -0.0033825631253421307, -0.018505893647670746, 0.011260662227869034]
The embedding vector has 256 values
[-0.0933709368109703, 0.09756065160036087, 0.06314508616924286, '...', 0.02887915074825287, 0.10294743627309799, 0.0029365457594394684]
The embedding vector has 256 values
[-0.13000746071338654, -0.028471633791923523, 0.013390768319368362, '...', 0.014170813374221325, -0.11804677546024323, -0.0012513218680396676]
The embedding vector has 256 values
[-0.11348896473646164, 0.10475904494524002, -0.010851776227355003, '...', 0.0068202503

In [8]:
df_groceries['embed_data']

0      [-0.09502902626991272, 0.10975183546543121, -0...
1      [-0.042328130453825, 0.0757022351026535, 0.006...
2      [-0.12640845775604248, 0.06689798086881638, -0...
3      [-0.03100176900625229, 0.1132238507270813, -0....
4      [-0.0955319032073021, 0.0772896334528923, 0.01...
                             ...                        
127    [-0.22361263632774353, 0.03060101717710495, -0...
128    [-0.13619254529476166, 0.16665665805339813, 0....
129    [-0.049682989716529846, 0.06974133849143982, 0...
130    [-0.11458425223827362, 0.09352097660303116, -0...
131    [-0.10596717894077301, 0.005986126605421305, -...
Name: embed_data, Length: 132, dtype: object

In [9]:
def saving_embedding_data_groceries_types(id,embedding_vector):
        
        # Convert the embedding vector to bytes
        embedding_vector_bytes = pickle.dumps(embedding_vector)
        
        # Define the SQL update statement
        update_sql = f"""
        UPDATE groceries_types
        SET embedding_vector = %s
        WHERE grocery_type_id = %s
        """
        
        # Execute the update statement (replace `1` with the appropriate row ID)
        cursor.execute(update_sql, (embedding_vector_bytes, id))
        
        # Commit the changes
        connection.commit()

In [10]:
def saving_embedding_data_recipes(id,embedding_vector):
        
        # Convert the embedding vector to bytes
        embedding_vector_bytes = pickle.dumps(embedding_vector)
        
        # Define the SQL update statement
        update_sql = f"""
        UPDATE recipes
        SET embedding_vector = %s
        WHERE recipe_id = %s
        """
        
        # Execute the update statement (replace `1` with the appropriate row ID)
        cursor.execute(update_sql, (embedding_vector_bytes, id))
        
        # Commit the changes
        connection.commit()

In [11]:
for id in df_recipes['recipe_id']:
    embedding_vector = df_recipes.loc[df_recipes['recipe_id'] == id].embed_data
    saving_embedding_data_recipes(id,embedding_vector)  

In [12]:
select_sql = f"SELECT grocery_type_id, embedding_vector FROM groceries_types;"

# Execute the query
cursor.execute(select_sql)

# Fetch all results
results = cursor.fetchall()

In [13]:
def saving_embedding_data_groceries(id, embedding_vector):
    # Convert the embedding vector to bytes
    embedding_vector_bytes = pickle.dumps(embedding_vector)
    
    # Define the SQL update statement
    update_sql = f"""
    UPDATE groceries_types
    SET embedding_vector = %s
    WHERE grocery_type_id = %s
    """
    
    # Execute the update statement
    cursor.execute(update_sql, (embedding_vector_bytes, id))
    
    # Commit the changes
    connection.commit()

# Loop over each row in df_groceries
for id in df_groceries['grocery_type_id']:
    embedding_vector = df_groceries.loc[df_groceries['grocery_type_id'] == id, 'embed_data'].values[0]
    print(embedding_vector)
    saving_embedding_data_groceries(id, embedding_vector)

[-0.09502903  0.10975184 -0.06067581  0.01104211  0.06513727  0.03078405
  0.05978352 -0.09859819  0.05978352 -0.12938224  0.113321    0.06959873
 -0.02732642  0.01572664 -0.02899947 -0.07673705  0.05933737  0.1026135
  0.00370859  0.04572993  0.01449973 -0.033684   -0.04506071  0.02565338
 -0.00758448 -0.0312302  -0.00071453  0.10216736  0.13205911 -0.01940734
  0.07762934  0.08298309  0.01383052  0.00088532  0.09101371  0.14812036
  0.04483764 -0.00593931  0.00296966  0.045953    0.00914599 -0.03346093
 -0.00096897  0.01037289  0.02029963 -0.00462876  0.02163807  0.13652058
  0.06246039  0.05978352  0.08565997  0.02375726  0.04126848  0.00451723
 -0.0278841   0.15882786 -0.0217496   0.03011483  0.02442648 -0.1084134
  0.05286826  0.0278841  -0.02821871 -0.1231362  -0.0180689  -0.04952217
 -0.01031712  0.01784583 -0.12938224 -0.11421329 -0.01940734 -0.06022967
  0.01583817  0.03881467  0.01751122  0.11510558  0.06335268  0.09012143
 -0.06201425  0.12224391  0.06469112  0.03390707  0.0

In [14]:

# Commit the changes
connection.commit()

# Close the cursor and connection
cursor.close()
connection.close()

: 